In [1]:
# not used
import gevent.monkey

gevent.monkey.patch_all()

from locust import HttpUser, task
from locust.env import Environment
from locust.stats import stats_printer, stats_history
from locust.log import setup_logging
from locust.runners import STATE_STOPPED
import gevent
import time
import csv
import re
import json
import requests
from datetime import datetime, timedelta
from prometheus_api_client import PrometheusConnect
import numpy as np
import os
from collections import defaultdict


PROM_URLS = [
    "http://yl-01.lab.uvalight.net:31119",
    "http://yl-02.lab.uvalight.net:31119",
    "http://yl-03.lab.uvalight.net:31119",
    "http://yl-04.lab.uvalight.net:31119",
    "http://yl-06.lab.uvalight.net:31119"
]

NODES = [
    "yl-01.lab.uvalight.net",
    "yl-02.lab.uvalight.net",
    "yl-03.lab.uvalight.net",
    "yl-04.lab.uvalight.net",
    "yl-06.lab.uvalight.net"
]

FUNCTION_MAP = {
    "basic": "matrix-multiplication",
    "data_local": "image-resize",
    "dag": "task1"
}

# ===================== 请求日志缓存 =====================
entry_logs = []
warmup_done_time = 0

def count_offloads(resp):
    if not isinstance(resp, dict):
        return 0
    count = 0
    msg = resp.get("message", "")
    if isinstance(msg, str) and re.search(r"Offload(ed)?\s+to", msg, re.IGNORECASE):
        count += 1
    nested = resp.get("response")
    if isinstance(nested, dict):
        count += count_offloads(nested)
    return count


class FaaSUser(HttpUser):
    @task
    def call_function(self):
        payload = {
            "fn_name": FUNCTION_MAP.get(os.getenv("FUNC_TYPE", "basic"), "matrix-multiplication"),
            "payload": None
        }
        if os.getenv("FUNC_TYPE") == "data_local":
            payload["payload"] = "http://yl-01.lab.uvalight.net:9000/images/image.jpg"

        start = time.time()
        with self.client.post("/entry", json=payload, catch_response=True) as response:
            try:
                resp_json = response.json()
                entry_time = resp_json.get("total_time", -1)
                architecture = resp_json.get("architecture", "dynamic")
                flattened = json.dumps(resp_json)
                match = re.search(r"Total time.*?([\d\.]+)", flattened)
                # print(match.group(1))
                exec_time = float(match.group(1)) if match else None
                offload_cnt = count_offloads(resp_json)
                if response.status_code == 500:
                    print(response.text)
                # entry_logs.append({
                #     "status_code": response.status_code,
                #     "total_time": float(entry_time),
                #     "exec_time": exec_time,
                #     "architecture": architecture,
                #     "offload_cnt": offload_cnt
                # })
                if time.time() >= warmup_done_time:
                    entry_logs.append({
                        "status_code": response.status_code,
                        "total_time": float(entry_time),
                        "exec_time": exec_time,
                        "architecture": architecture,
                        "offload_cnt": offload_cnt
                    })

                response.success()

            except Exception as e:
                response.failure(f"Exception: {e}")


# ===================== Prometheus 查询 =====================
def parse_datetime(ts):
    return datetime.utcfromtimestamp(ts)


def create_prom_clients():
    return [PrometheusConnect(url=url, disable_ssl=True) for url in PROM_URLS]


def cpu_avg_utilization(prom, start_time, end_time):
    query = """
    100 - (avg by(instance) (rate(node_cpu_seconds_total{mode="idle"}[30s])) * 100)
    """

    metric_data = prom.custom_query_range(
        query=query,
        start_time=start_time,
        end_time=end_time,
        step='15s'
    )
    result = {}
    for data in metric_data:
        instance = data['metric']['instance'].split(':')[0]
        values = [float(v[1]) for v in data['values'] if float(v[1]) > 0]
        result[instance] = sum(values) / len(values) * 100 if values else 0.0
    return result


def memory_avg_utilization(prom, start_time, end_time):
    query = "node_memory_MemTotal_bytes - (node_memory_MemFree_bytes + node_memory_Cached_bytes + node_memory_Buffers_bytes)"
    metric_data = prom.custom_query_range(
        query=query,
        start_time=start_time,
        end_time=end_time,
        step='15s'
    )
    result = {}
    for data in metric_data:
        instance = data['metric']['instance'].split(':')[0]
        values = [float(v[1]) for v in data['values']]
        result[instance] = np.mean(values) / 1024 / 1024 if values else 0.0
    return result


# ===================== 控制逻辑 =====================
def reload_architecture(arch):
    for node in NODES:
        try:
            res = requests.post(f"http://{node}:31113/reload", json={"architecture": arch}, timeout=5)
            print(f"[✓] Reloaded {node} to {arch}: {res.status_code}")
        except Exception as e:
            print(f"[x] Reload error on {node}: {e}")


def run_experiment_stage(host, func_type, arch, users, spawn_rate, duration, output_csv, metrics_output, unit_data):
    global entry_logs
    entry_logs = []

    os.environ["FUNC_TYPE"] = func_type
    reload_architecture(arch)
    print(f"🔄 Switched architecture to {arch} | function: {func_type}")

    setup_logging("INFO", None)
    env = Environment(user_classes=[FaaSUser])
    env.create_local_runner()
    env.host = host

    gevent.spawn(stats_printer, env.stats)
    gevent.spawn(stats_history, env.runner)

    # start_time = parse_datetime(time.time())
    start_time = datetime.utcnow() + timedelta(seconds=30)
    global warmup_done_time
    warmup_done_time = time.time() + 30 
    env.runner.start(user_count=users, spawn_rate=spawn_rate)
    gevent.spawn_later(duration, lambda: env.runner.quit())

    while env.runner.state != STATE_STOPPED:
        time.sleep(1)
    # end_time = parse_datetime(time.time())
    end_time = datetime.utcnow()
    
    NODE_TO_UNIT = {
        "yl-04": "cloud-1",
        "yl-05": "cloud-2",
        "yl-01.lab.uvalight.net": "edge-A1",
        "yl-02.lab.uvalight.net": "edge-A2",
        "yl-03.lab.uvalight.net": "edge-B1",
        "yl-06.lab.uvalight.net": "edge-B2"
    }
    
    unit_stats = defaultdict(lambda: {"cpu": [], "mem": []})
    
    prom_clients = create_prom_clients()
    for prom in prom_clients:
        cpu_query = '100 - (avg by(instance) (rate(node_cpu_seconds_total{mode="idle"}[30s])) * 100)'
        cpu_data = prom.custom_query_range(
            query=cpu_query,
            start_time=start_time,
            end_time=end_time,
            step='15s'
        )
        for d in cpu_data:
            node = d["metric"]["instance"].split(":")[0]
            unit = NODE_TO_UNIT.get(node)
            values = [float(v[1]) for v in d["values"] if float(v[1]) > 0]
            if unit and values:
                unit_stats[unit]["cpu"].append(np.mean(values))
    
        mem_query = "node_memory_MemTotal_bytes - (node_memory_MemFree_bytes + node_memory_Cached_bytes + node_memory_Buffers_bytes)"
        mem_data = prom.custom_query_range(
            query=mem_query,
            start_time=start_time,
            end_time=end_time,
            step='15s'
        )
        for d in mem_data:
            node = d["metric"]["instance"].split(":")[0]
            unit = NODE_TO_UNIT.get(node)
            values = [float(v[1]) for v in d["values"] if float(v[1]) > 0]
            if unit and values:
                unit_stats[unit]["mem"].append(np.mean(values) / 1024 / 1024)
    

    with open(unit_data, "a", newline="") as f:
        fieldnames = ["fn_type", "architecture", "concurrency", "faas_unit", "avg_cpu", "avg_mem_MB"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if f.tell() == 0:
            writer.writeheader()
    
        for unit, stats in unit_stats.items():
            writer.writerow({
                "fn_type": func_type,
                "architecture": arch,
                "concurrency": users,
                "faas_unit": unit,
                "avg_cpu": round(np.mean(stats["cpu"]), 2) if stats["cpu"] else None,
                "avg_mem_MB": round(np.mean(stats["mem"]), 2) if stats["mem"] else None,
            })

    cpu_all = [cpu_avg_utilization(p, start_time, end_time) for p in prom_clients]
    mem_all = [memory_avg_utilization(p, start_time, end_time) for p in prom_clients]

    flat_cpu = [v for d in cpu_all for v in d.values()]
    flat_mem = [v for d in mem_all for v in d.values()]
    cpu_avg = sum(flat_cpu) / len(flat_cpu) if flat_cpu else None
    mem_avg = sum(flat_mem) / len(flat_mem) if flat_mem else None

    with open(output_csv, "w", newline="") as f:
        fieldnames = ["fn_type", "architecture", "status_code", "total_time", "exec_time", "offload_cnt",
                      "avg_cpu_usage", "avg_mem_usage_MB"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in entry_logs:
            row["fn_type"] = func_type
            # row["architecture"] = architecture
            row["avg_cpu_usage"] = cpu_avg
            row["avg_mem_usage_MB"] = mem_avg
            writer.writerow(row)

    print(f"✅ Completed: {func_type}-{arch} | saved to {output_csv}")



D:\ProgramData\anaconda3\envs\sionna\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
host = "http://yl-01.lab.uvalight.net:31113"
duration = 180
# spawn rate and speed
concurrency_settings = [
    (2, 2),
    (4, 4),
    (6, 6),
    (8, 8),
    (10, 10),
    (12, 12),
    (15, 15),
    (20, 20),
    (30, 30),
    (40, 40),
    # (60, 60),
]

configs = [
    # ("data_local", "decentralized"),
    # ("data_local", "federated"),
    # ("data_local", "centralized"),
    ("data_local", "dynamic"),
    # ("basic", "decentralized"),
    # ("basic", "federated"),
    # ("basic", "centralized"),
    ("basic", "dynamic")
]

for users, spawn_rate in concurrency_settings:
    print(f"\n=== Starting test: {users} users @ {spawn_rate}/s ===")
    for func_type, arch in configs:
        os.makedirs(func_type, exist_ok=True)
        outfile = f"{func_type}/results_{arch}_{users}.csv"
        print(f"→ Running: func={func_type}, arch={arch}, users={users}")
        run_experiment_stage(
            host=host,
            func_type=func_type,
            arch=arch,
            users=users,
            spawn_rate=spawn_rate,
            duration=duration,
            output_csv=outfile,
            metrics_output=f"{func_type}/metrics_dynamic_{users}.csv" if arch == "dynamic" else None,
            unit_data = f"{func_type}/metrics_unit_{arch}_{users}.csv"
        )
        print("✔ Finished one configuration. Sleeping 60s...\n")
        time.sleep(60)


=== Starting test: 2 users @ 2/s ===
→ Running: func=data_local, arch=dynamic, users=2
[✓] Reloaded yl-01.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-02.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-03.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-04.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-06.lab.uvalight.net to dynamic: 200
🔄 Switched architecture to dynamic | function: data_local


[2025-06-23 17:03:10,306] Lyt2023/INFO/locust.runners: Ramping to 2 users at a rate of 2.00 per second
[2025-06-23 17:03:10,306] Lyt2023/INFO/locust.runners: All users spawned: {"FaaSUser": 2} (2 total users)


✅ Completed: data_local-dynamic | saved to data_local/results_dynamic_2.csv
✔ Finished one configuration. Sleeping 60s...
→ Running: func=basic, arch=dynamic, users=2
[✓] Reloaded yl-01.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-02.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-03.lab.uvalight.net to dynamic: 200


[2025-06-23 17:07:13,270] Lyt2023/INFO/locust.runners: Ramping to 2 users at a rate of 2.00 per second
[2025-06-23 17:07:13,270] Lyt2023/INFO/locust.runners: All users spawned: {"FaaSUser": 2} (2 total users)


[✓] Reloaded yl-04.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-06.lab.uvalight.net to dynamic: 200
🔄 Switched architecture to dynamic | function: basic
✅ Completed: basic-dynamic | saved to basic/results_dynamic_2.csv
✔ Finished one configuration. Sleeping 60s...

=== Starting test: 4 users @ 4/s ===
→ Running: func=data_local, arch=dynamic, users=4
[✓] Reloaded yl-01.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-02.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-03.lab.uvalight.net to dynamic: 200


...

[✓] Reloaded yl-06.lab.uvalight.net to dynamic: 200
🔄 Switched architecture to dynamic | function: basic
✅ Completed: basic-dynamic | saved to basic/results_dynamic_30.csv
✔ Finished one configuration. Sleeping 60s...

=== Starting test: 40 users @ 40/s ===
→ Running: func=data_local, arch=dynamic, users=40
[✓] Reloaded yl-01.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-02.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-03.lab.uvalight.net to dynamic: 200


[2025-06-23 18:15:44,895] Lyt2023/INFO/locust.runners: Ramping to 40 users at a rate of 40.00 per second
[2025-06-23 18:15:44,897] Lyt2023/INFO/locust.runners: All users spawned: {"FaaSUser": 40} (40 total users)


[✓] Reloaded yl-04.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-06.lab.uvalight.net to dynamic: 200
🔄 Switched architecture to dynamic | function: data_local
✅ Completed: data_local-dynamic | saved to data_local/results_dynamic_40.csv
✔ Finished one configuration. Sleeping 60s...
→ Running: func=basic, arch=dynamic, users=40
[✓] Reloaded yl-01.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-02.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-03.lab.uvalight.net to dynamic: 200


[2025-06-23 18:19:46,422] Lyt2023/INFO/locust.runners: Ramping to 40 users at a rate of 40.00 per second
[2025-06-23 18:19:46,423] Lyt2023/INFO/locust.runners: All users spawned: {"FaaSUser": 40} (40 total users)


[✓] Reloaded yl-04.lab.uvalight.net to dynamic: 200
[✓] Reloaded yl-06.lab.uvalight.net to dynamic: 200
🔄 Switched architecture to dynamic | function: basic
✅ Completed: basic-dynamic | saved to basic/results_dynamic_40.csv
✔ Finished one configuration. Sleeping 60s...


In [3]:
import pandas as pd
import glob
import os

def merge_results_by_architecture(folder):
    arch_list = ["centralized", "federated", "decentralized", "dynamic"]
    arch_list = ["dynamic"]
    for arch in arch_list:
        pattern = os.path.join(folder, f"results_{arch}_*.csv")
        files = sorted(glob.glob(pattern))

        df_list = []
        for file in files:
            df = pd.read_csv(file)
            df["concurrency"] = int(file.split("_")[-1].replace(".csv", ""))
            df["architecture"] = arch
            df_list.append(df)

        if df_list:
            combined = pd.concat(df_list, ignore_index=True)
            output_file = os.path.join(folder, f"results_{arch}_all.csv")
            combined.to_csv(output_file, index=False)
            print(f"✅ Saved: {output_file}")
        else:
            print(f"⚠️ No files found for architecture: {arch}")

merge_results_by_architecture("basic")
merge_results_by_architecture("data_local")

✅ Saved: basic\results_dynamic_all.csv
✅ Saved: data_local\results_dynamic_all.csv


D:\ProgramData\anaconda3\envs\sionna\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
